In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [3]:
# Load our dataframe
df = pd.read_csv('conspires.csv', index_col='id')


# Get our two columns of texts
bodies = df.body.tolist()
comments = df.comments.tolist()

# Join them, clean them of NaNs, and make them intro a string
joined = [*bodies, *comments]  # unpack both iterables in a list literal
cleaned = [x for x in joined if str(x) != 'nan'] # get rid of the nans
text = ' '.join(cleaned) # make it one long string

print(len(text))
print(text[0:100])

1171824
Please provide your feedback on the percentage of food products at your local grocery store that con


We have a data set that is slightly larger than the Tiny Shakespeare corpus.

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|~ £°³·×äèéíñóöāōūʻАВЕКМНОРСТаеорсухְֱֲִֵֶַָֹֻּׁׂאבדהוחיךכלםמןנסעפץצקרשת‑–—‘’“”•…≠♀♂♄✨❤。שׂבּכּלּתּוֹ️￼𝘢𝘤𝘭𝘵𝘶𝘺🃏🇨🇩🇪🇮🇱🇳🇷🇸🇺🌍🌏🍻🎉🎩🎯🏆🏻🏽🏿🐑🐒👀👆👌👍👏👨💀💉💕💩💫💯🕋😁😂😄😅😆😇😈😉😊😌😏😔😕😘😭🙂🙃🙄🙈🙉🙌🙏🤓🤔🤖🤠🤡🤢🤣🤦🤧🤪🤭🤮🤯🤷🥁🥪🥴🥵🧐🧢
272


In [5]:
# our nanoGPT neural network will be based on characters
# chatGPT works on parts of words
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] #encodes string to chars to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #takes list of integers and outputs string

In [6]:
encode("hello there")

[73, 70, 77, 77, 80, 1, 85, 73, 70, 83, 70]

In [7]:
decode([73, 70, 77, 77, 80, 1, 85, 73, 70, 83, 70])

'hello there'

The smaller the vocabularyused for encoding, the bigger the eventual encoding. See Open AI's `tiktoken` library which has a 50k vocabulary and would encode "hello there" into three tokens."

In [8]:
# we now econde the entire dataset above and store it in a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1171824]) torch.int64
tensor([49, 77, 70, 66, 84, 70,  1, 81, 83, 80, 87, 74, 69, 70,  1, 90, 80, 86,
        83,  1, 71, 70, 70, 69, 67, 66, 68, 76,  1, 80, 79,  1, 85, 73, 70,  1,
        81, 70, 83, 68, 70, 79, 85, 66, 72, 70,  1, 80, 71,  1, 71, 80, 80, 69,
         1, 81, 83, 80, 69, 86, 68, 85, 84,  1, 66, 85,  1, 90, 80, 86, 83,  1,
        77, 80, 68, 66, 77,  1, 72, 83, 80, 68, 70, 83, 90,  1, 84, 85, 80, 83,
        70,  1, 85, 73, 66, 85,  1, 68, 80, 79])


In [9]:
# Split into train/test
cut = int(0.9*len(data))
train_data = data[:cut]
test_data  = data[cut:]

Training on the entire corpus all at once is computationally expensive, so most programs break the corpus into "chunks." Here we're just calling such a chunk `block_size`.

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([49, 77, 70, 66, 84, 70,  1, 81, 83])

What this means for training our neural network is this:

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}.")

When input is tensor([49]), the target is 77.
When input is tensor([49, 77]), the target is 70.
When input is tensor([49, 77, 70]), the target is 66.
When input is tensor([49, 77, 70, 66]), the target is 84.
When input is tensor([49, 77, 70, 66, 84]), the target is 70.
When input is tensor([49, 77, 70, 66, 84, 70]), the target is 1.
When input is tensor([49, 77, 70, 66, 84, 70,  1]), the target is 81.
When input is tensor([49, 77, 70, 66, 84, 70,  1, 81]), the target is 83.


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x, y

In [13]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[ 1, 58, 80, 86, 13,  1, 78, 86],
        [ 8, 13,  1,  3, 42,  1, 77, 74],
        [85, 73, 66, 85,  1, 74, 85,  1],
        [88, 70, 77, 77,  1, 66, 84,  1]])
targets:
torch.Size([4, 8])
tensor([[58, 80, 86, 13,  1, 78, 86, 77],
        [13,  1,  3, 42,  1, 77, 74, 79],
        [73, 66, 85,  1, 74, 85,  1, 78],
        [70, 77, 77,  1, 66, 84,  1, 80]])


Batch, Time, and Channel is actually batch, block, by vocabulary.

In [14]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off logits for the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)
        
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)